In [ ]:
import pandas as pd

In [ ]:
#!/usr/bin/env python
# coding: utf-8
#######################################################################
# How to setup AndroSensor:
# Install AndroSensor
# Ensure when installing, allow app to run in background.
# Open settings and change recording interval to fast.
#
#
#
#######################################################################
import glob
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'
import numpy as np
import scipy as sp
import scipy.signal as sg

px.set_mapbox_access_token(open(".mapbox_token").read())

In [ ]:
# IF useing ; sep, then delete the first line.
path = r'phone_data'  # use your path
all_files = glob.glob(os.path.join(path, "*.csv")) 

df_from_each_file = [pd.read_csv(f, sep=";") for f in all_files]
concatenated_df = pd.concat(df_from_each_file, ignore_index=True, sort=True)

# Convert datatime to actual datetime
concatenated_df["timestamp"] = pd.to_datetime(concatenated_df["YYYY-MO-DD HH-MI-SS_SSS"],format='%Y-%m-%d %H:%M:%S:%f')
concatenated_df["Speed(km/h)"] = concatenated_df["LOCATION Speed ( Kmh)"]

In [ ]:
# calculate abs accleration
concatenated_df["abs_acceleration"] = np.linalg.norm(
    concatenated_df[["ACCELEROMETER X (m/s²)", "ACCELEROMETER Y (m/s²)", "ACCELEROMETER Z (m/s²)"]].values, axis=1)

# Calculate abs location
concatenated_df["abs_loc"] = np.linalg.norm(concatenated_df[['LOCATION Latitude : ', 'LOCATION Longitude : ']].values,
                                            axis=1)

# Parse GPS sats properly
concatenated_df['Satellites in range'] = concatenated_df['Satellites in range'].str.extract(r'(\d*) \/ \d*')
concatenated_df['Satellites in range'] = pd.to_numeric(concatenated_df['Satellites in range'] )

In [ ]:
concatenated_df = concatenated_df[~(concatenated_df['timestamp'] < '2021-03-12 18:46:00')]
concatenated_df = concatenated_df[~(concatenated_df['timestamp'] > '2021-03-12 18:52:00')]

# Display 3-axis acceleration vs time

In [ ]:
# Display accelration
fig = px.line(concatenated_df,
              x="timestamp",
              y=["ACCELEROMETER X (m/s²)", "ACCELEROMETER Y (m/s²)", "ACCELEROMETER Z (m/s²)"],
              title='Acceleration'
             )


In [ ]:
fig.show()

# Parse data from ebike datalogger

In [ ]:
my_cols = ["ts", "sensor", "sensor_no", "V_bus[V]", "V_shunt[mV]","Current[mA]", "Power[mW]"]
df = pd.read_csv("data.csv",
            names=my_cols,
            engine='c')

In [ ]:
df.head()

In [ ]:
df['Datetime'] = pd.to_datetime(df["ts"])
df.sort_values(by='Datetime',inplace = True)

df = df[~(df['Datetime'] < '2021-03-12 18:46:00')]
df = df[~(df['Datetime'] > '2021-03-12 18:52:00')]

#df = df.set_index('Datetime')

In [ ]:
df.dtypes

In [ ]:
# Using plotly.express
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'browser'

fig = px.line(df, x = "Datetime", y =["V_bus[V]","V_shunt[mV]","Current[mA]","Power[mW]"])
fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df["Datetime"],
    y=df["V_bus[V]"],
    name="V_bus[V]"
))


fig.add_trace(go.Scatter(
    x=df["Datetime"],
    y=df["V_shunt[mV]"],
    name="V_shunt[mV]",
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=df["Datetime"],
    y=df["Current[mA]"],
    name="Current[mA]",
    yaxis="y3"
))

fig.add_trace(go.Scatter(
    x=df["Datetime"],
    y=df["Power[mW]"],
    name="Power[mW]",
    yaxis="y4"
))


# Create axis objects
fig.update_layout(
    yaxis=dict(
        title="V_bus[V]",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),
    yaxis2=dict(
        title="V_shunt[mV]",
        titlefont=dict(
            color="#ff7f0e"
        ),
        tickfont=dict(
            color="#ff7f0e"
        ),
        anchor="free",
        overlaying="y",
        side="left",
        position=0.05
    ),
    yaxis3=dict(
        title="Current[mA]",
        titlefont=dict(
            color="#d62728"
        ),
        tickfont=dict(
            color="#d62728"
        ),
        anchor="x",
        overlaying="y",
        side="right"
    ),
    yaxis4=dict(
        title="Power[mW]",
        titlefont=dict(
            color="#9467bd"
        ),
        tickfont=dict(
            color="#9467bd"
        ),
        anchor="free",
        overlaying="y",
        side="right",
        position=0.95
    )
)

# Update layout properties
fig.update_layout(
    title_text="Power, shunt voltage, battery voltage and current",
)

fig.write_html("power.html")


fig.show()